In [1]:
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [2]:
toolset_config = {
    'tools': {
        'directory-tree': {
            'description': "Generate a directory tree with standard exclusions and gitignore support",
            'execution': {
                'command': 'tree <<directory>> -a --gitignore -I ".git|.claude|.env|.venv|env|node_modules|__pycache__|.DS_Store|*.pyc<<custom_excludes>>" <<format_args>>',  # noqa: E501
            },
            'parameters': {
                'directory': {
                    'description': "Directory to generate tree for",
                    "required": True,
                },
                'custom_excludes': {
                    'description': "Additional patterns to exclude (begin with | e.g., '|build|dist')",
                    'required': False,
                },
                'format_args': {
                    'description': "Additional formatting arguments (e.g., '-L 3 -C --dirsfirst')",
                    'required': False,
                },
            },
        },
        'extract-html-text': {
            'description': "Fetch a webpage and extract pure text, removing HTML tags",
            'execution': {
                'command': "curl -s <<url>> | sed '/<style/,/<\\/style>/d; /<script/,/<\\/script>/d' | sed 's/<[^>]*>//g' | grep -v '^$' | sed 's/^[[:space:]]*//' | sed '/^[[:space:]]*$/d' | sed -e ':a' -e 'N' -e '$!ba' -e 's/\\n\\n\\n*/\\n\\n/g'", 
            },
            'parameters': {
                'url': {
                'description': "URL of the webpage to fetch",
                'required': True,
                },
            },
        },
    },
}
# convert to json string
toolset_config_json = json.dumps(toolset_config)
print(json.dumps(toolset_config_json))

"{\"tools\": {\"directory-tree\": {\"description\": \"Generate a directory tree with standard exclusions and gitignore support\", \"execution\": {\"command\": \"tree <<directory>> -a --gitignore -I \\\".git|.claude|.env|.venv|env|node_modules|__pycache__|.DS_Store|*.pyc<<custom_excludes>>\\\" <<format_args>>\"}, \"parameters\": {\"directory\": {\"description\": \"Directory to generate tree for\", \"required\": true}, \"custom_excludes\": {\"description\": \"Additional patterns to exclude (begin with | e.g., '|build|dist')\", \"required\": false}, \"format_args\": {\"description\": \"Additional formatting arguments (e.g., '-L 3 -C --dirsfirst')\", \"required\": false}}}, \"extract-html-text\": {\"description\": \"Fetch a webpage and extract pure text, removing HTML tags\", \"execution\": {\"command\": \"curl -s <<url>> | sed '/<style/,/<\\\\/style>/d; /<script/,/<\\\\/script>/d' | sed 's/<[^>]*>//g' | grep -v '^$' | sed 's/^[[:space:]]*//' | sed '/^[[:space:]]*$/d' | sed -e ':a' -e 'N' 

In [3]:
server_params = StdioServerParameters(
    command='python',
    # args=['-m', 'mcp_this', '--tools_path', 'toolset_example__curl.yaml'],
    args=['-m', 'mcp_this', '--tools', json.dumps(toolset_config)],
)

async with stdio_client(server_params) as (read, write):  # noqa: SIM117
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()
        print("Available tools:\n-----------------------\n")
        for tool in tools.tools:
            print(f"Tool: `{tool.name}`:\n=============\n")
            print(f"{tool.description}\n-----------------------\n")

        dir_tree_result = await session.call_tool(
            'directory-tree',
            {'directory': '..'},
        )
        extract_text_result = await session.call_tool(
            'extract-html-text',
            {'url': 'https://example.com'},
        )

Available tools:
-----------------------

Tool: `directory-tree`:

TOOL DESCRIPTION:

Generate a directory tree with standard exclusions and gitignore support

COMMAND CALLED:

`tree <<directory>> -a --gitignore -I ".git|.claude|.env|.venv|env|node_modules|__pycache__|.DS_Store|*.pyc<<custom_excludes>>" <<format_args>>`

Text like <<parameter_name>> (e.g. <<directory>>) will be replaced with parameter values.

PARAMETERS:

- directory [REQUIRED] (string): Directory to generate tree for
- custom_excludes [OPTIONAL] (string): Additional patterns to exclude (begin with | e.g., '|build|dist')
- format_args [OPTIONAL] (string): Additional formatting arguments (e.g., '-L 3 -C --dirsfirst')
-----------------------

Tool: `extract-html-text`:

TOOL DESCRIPTION:

Fetch a webpage and extract pure text, removing HTML tags

COMMAND CALLED:

`curl -s <<url>> | sed '/<style/,/<\/style>/d; /<script/,/<\/script>/d' | sed 's/<[^>]*>//g' | grep -v '^$' | sed 's/^[[:space:]]*//' | sed '/^[[:space:]]*$/d'

In [4]:
# last tool
tool

Tool(name='extract-html-text', description="TOOL DESCRIPTION:\n\nFetch a webpage and extract pure text, removing HTML tags\n\nCOMMAND CALLED:\n\n`curl -s <<url>> | sed '/<style/,/<\\/style>/d; /<script/,/<\\/script>/d' | sed 's/<[^>]*>//g' | grep -v '^$' | sed 's/^[[:space:]]*//' | sed '/^[[:space:]]*$/d' | sed -e ':a' -e 'N' -e '$!ba' -e 's/\\n\\n\\n*/\\n\\n/g'`\n\nText like <<parameter_name>> (e.g. <<url>>) will be replaced with parameter values.\n\nPARAMETERS:\n\n- url [REQUIRED] (string): URL of the webpage to fetch", inputSchema={'properties': {'url': {'title': 'url', 'type': 'string'}}, 'required': ['url'], 'title': 'extract_html_textArguments', 'type': 'object'}, annotations=None)

In [5]:
dir_tree_result

CallToolResult(meta=None, content=[TextContent(type='text', text='..\n├── .github\n│\xa0\xa0 └── workflows\n│\xa0\xa0     └── tests.yaml\n├── .gitignore\n├── .ruff.toml\n├── LICENSE\n├── Makefile\n├── README.md\n├── examples\n│\xa0\xa0 ├── claude_desktop_config_default.json\n│\xa0\xa0 ├── cli.py\n│\xa0\xa0 ├── examples.ipynb\n│\xa0\xa0 ├── mcp_config_cli.json\n│\xa0\xa0 ├── toolset_example__curl.yaml\n│\xa0\xa0 └── top_level_tools_example.yaml\n├── pyproject.toml\n├── src\n│\xa0\xa0 └── mcp_this\n│\xa0\xa0     ├── __init__.py\n│\xa0\xa0     ├── __main__.py\n│\xa0\xa0     ├── configs\n│\xa0\xa0     │\xa0\xa0 ├── default.yaml\n│\xa0\xa0     │\xa0\xa0 ├── default_archive.yaml\n│\xa0\xa0     │\xa0\xa0 └── example.yaml\n│\xa0\xa0     ├── mcp_server.py\n│\xa0\xa0     └── tools.py\n├── tests\n│\xa0\xa0 ├── fixtures\n│\xa0\xa0 │\xa0\xa0 └── test_config.yaml\n│\xa0\xa0 ├── test_command_execution_edge_cases.py\n│\xa0\xa0 ├── test_config_files.py\n│\xa0\xa0 ├── test_config_loading.py\n│\xa0\xa0 ├

In [6]:
print(dir_tree_result.content[0].text)

..
├── .github
│   └── workflows
│       └── tests.yaml
├── .gitignore
├── .ruff.toml
├── LICENSE
├── Makefile
├── README.md
├── examples
│   ├── claude_desktop_config_default.json
│   ├── cli.py
│   ├── examples.ipynb
│   ├── mcp_config_cli.json
│   ├── toolset_example__curl.yaml
│   └── top_level_tools_example.yaml
├── pyproject.toml
├── src
│   └── mcp_this
│       ├── __init__.py
│       ├── __main__.py
│       ├── configs
│       │   ├── default.yaml
│       │   ├── default_archive.yaml
│       │   └── example.yaml
│       ├── mcp_server.py
│       └── tools.py
├── tests
│   ├── fixtures
│   │   └── test_config.yaml
│   ├── test_command_execution_edge_cases.py
│   ├── test_config_files.py
│   ├── test_config_loading.py
│   ├── test_main.py
│   ├── test_mcp_server.py
│   └── test_tool_registration.py
└── uv.lock

9 directories, 28 files



In [7]:
print(extract_text_result.content[0].text)

Example Domain
Example Domain
This domain is for use in illustrative examples in documents. You may use this
domain in literature without prior coordination or asking for permission.
More information...

